# Investment Problem (EHL Problem 9.1)

* A microelectronics manufacturing facility is considering six projects to improve operations as well as profitability. Due to expenditure limitations and engineering staffing constraints, however, not all of these projects can be implemented. 
* The resource limitations are
  * First-year expenditure: USD 450,000
  * Second-year expenditure: USD 400,000
  * Engineering hours: 10,000

* A new or modernized production line must be provided (project 1 or 2). Automation is feasible only for the new line. Either project 5 or project 6 can be selected, but not both. Determine which projects maximize the net present value subject to the various constraints.

|Project|Year 1 Exp.|Year 2 Exp.|Engg hrs|NPV     |
|-------|----------:|----------:|-------:|-------:|
|1      |300,000    |0          | 4,000  | 100,000|
|2      |100,000    |300,000    | 7,000  | 150,000|
|3      |0          |200,000    | 2,000  |  35,000|
|4      |50,000     |100,000    | 6,000  |  75,000|
|5      |50,000     |300,000    | 3,000  | 125,000|
|6      |100,000    |200,000    |   600  |  60,000|

In [2]:
import pandas as pd
import pyomo.environ as pyo

df_data = pd.read_excel('investment_ex.xlsx', sheet_name='Sheet1', header=0, index_col=0)
options = df_data.index.tolist()
data = df_data.to_dict(orient='records')

print(data[1]['NPV'])

model = pyo.ConcreteModel()
model.select = pyo.Var(options,within=pyo.Binary)

def objRule(model):
    return sum(model.select[i]*data[i-1]['NPV'] for i in options)
model.obj = pyo.Objective(rule=objRule, sense=pyo.maximize)

def yr1Rule(model):
    return sum(model.select[i]*data[i-1]['Year1'] for i in options) <= 450
model.yr1Cons = pyo.Constraint(rule=yr1Rule)

def yr2Rule(model):
    return sum(model.select[i]*data[i-1]['Year2'] for i in options) <= 400
model.yr2Cons = pyo.Constraint(rule=yr2Rule)

def hrRule(model):
    return sum(model.select[i]*data[i-1]['Engg hrs'] for i in options) <= 10
model.hrsCons = pyo.Constraint(rule=hrRule)

model.prodline = pyo.Constraint(expr= model.select[1]+model.select[2] <= 1)
model.autoline = pyo.Constraint(expr= model.select[2]-model.select[3] >= 0)
model.waste = pyo.Constraint(expr= model.select[5]+model.select[6] <= 1)

#model.pprint()
opt = pyo.SolverFactory('glpk')
results = opt.solve(model)
model.display()

150
Model unknown

  Variables:
    select : Size=6, Index={1, 2, 3, 4, 5, 6}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   1.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 225.0

  Constraints:
    yr1Cons : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 350.0 : 450.0
    yr2Cons : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 300.0 : 400.0
    hrsCons : Size=1
        Key  : Lower : Body : Upper
        None :  None :  7.0 :  10.0
    prodline : Size=1
        Key  : Lower : Body : Upper
        None :  None 

# Plant Location (EHL Problem 9.14)

A plant location problem has arisen. Two possible sites exist for building a new plant, A and B, and two customer locations are to be supplied, C and D. Demands and production/supply costs are listed as follows.

Use the following notation to formulate the optimization problem, and solve it for the values of $I_1$ and $I_2$ as well as the values of $S_{ij}$. Each plant has a max capacity of 500 units per day.

* $I_i$ = decision variable (0-1) associated with the decision to build, or not to build, a plant in a given location, and thus incurs the associated fixed daily cost.
* $C_{ij}$ = unit cost of supplying customer $j$ from plant $i$
* $C_i$ = fixed daily cost of plant $i$
* $S_{ij}$ = quantity supplied from the $i$th plant to the $j$th customer
* $R_j$ = requirement of $j$th customer
* $Q_i$ = capacity of proposed plant

Production and transport costs per unit:
* A to C: USD 1.00
* A to D: USD 3.00
* B to C: USD 4.50
* B to D: USD 1.00

Fixed plant charges per day:
* A: USD 700
* B: USD 610

Minimum demand (units per day):
* C: 200
* D: 250

In [3]:
sources = ['A', 'B']
sinks = ['C', 'D']
Cij = {'A':{'C':1,'D':3},
       'B':{'C':4.5,'D':1,}}
Ci = {'A':700, 'B':610}
Rj = {'C':200, 'D':250}

m = pyo.ConcreteModel()
m.Ii = pyo.Var(sources,within=pyo.Binary)
m.Sij = pyo.Var(sources,sinks,within=pyo.NonNegativeReals)

def objRule(m):
    return sum(Ci[i]*m.Ii[i] for i in sources) \
        + sum(Cij[i][j]*m.Sij[i,j] for i in sources for j in sinks)
m.obj = pyo.Objective(rule=objRule, sense=pyo.minimize)

def supplyRule(m,sources):
    return sum(m.Sij[sources,j] for j in sinks) <= 500*m.Ii[sources]
m.supplyCons = pyo.Constraint(sources,rule=supplyRule)

def demandRule(m,sinks):
    return sum(m.Sij[i,sinks] for i in sources) >= Rj[sinks]
m.demandCons = pyo.Constraint(sinks,rule=demandRule)

#m.build1 = pyo.Constraint(expr= m.Ii['A']+m.Ii['B'] >= 1)

m.pprint()
opt = pyo.SolverFactory('glpk')
results = opt.solve(m)
m.display()

2 Var Declarations
    Ii : Size=2, Index={A, B}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          A :     0 :  None :     1 : False :  True : Binary
          B :     0 :  None :     1 : False :  True : Binary
    Sij : Size=4, Index={A, B}*{C, D}
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
        ('A', 'C') :     0 :  None :  None : False :  True : NonNegativeReals
        ('A', 'D') :     0 :  None :  None : False :  True : NonNegativeReals
        ('B', 'C') :     0 :  None :  None : False :  True : NonNegativeReals
        ('B', 'D') :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 700*Ii[A] + 610*Ii[B] + Sij[A,C] + 3*Sij[A,D] + 4.5*Sij[B,C] + Sij[B,D]

2 Constraint Declarations
    demandCons : Size=2, Index={C, D}, Active=True
        Key : Lower : Body                : Upper :